In [2]:
import os
import json
import requests
import random
from django.core.files.base import ContentFile
from shop.models import Category, Shop, Item
from urllib.parse import urljoin
from fake_useragent import UserAgent

In [32]:
class Yogiyo:
    HOST = 'https://www.yogiyo.co.kr'
    
    def __init__(self, headers):
        self.headers = headers
        
    def get_restaurant_list_by_geo(self, zip_code):
        url = self.HOST + '/api/v1/restaurants-geo/'
        params = dict(
            items=20,
            order='rank',
            page=0,
            search='',
            zip_code=zip_code,
        )
        res = requests.get(url, params=params, headers=self.headers)
        return res.json()
    
    def get_menu_list(self, restaurant_id):
        base_url = self.HOST + '/api/v1/restaurants/{restaurant_id}/menu/?add_photo_menu=original'
        url = base_url.format(restaurant_id=restaurant_id)
        res = requests.get(url, headers=self.headers)
        return res.json()

In [3]:
## 활용: 식당 등록

In [6]:
user_agent = UserAgent()

In [7]:
headers = {
    'X-ApiKey': 'iphoneap',
    'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
    'X-MOD-SBB-CTYPE': 'xhr',
    'User_Agent': user_agent.Chrome,
}

In [8]:
headers

{'X-ApiKey': 'iphoneap',
 'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
 'X-MOD-SBB-CTYPE': 'xhr',
 'User_Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1944.0 Safari/537.36'}

In [9]:
yogiyo = Yogiyo(headers)

In [12]:
zip_code = '130091'
restaurants = yogiyo.get_restaurant_list_by_geo('130091')['restaurants'] #pagination, restaurants
restaurants

[{'rating': 200060.959986042,
  'subtitle': '',
  'additional_discount_currently_active': False,
  'free_delivery_threshold': 0,
  'delivery_fee_explanation': '최소 주문 금액 13000원 이상',
  'is_available_delivery': True,
  'threshold': 8000,
  'adv_distance': None,
  'open': True,
  'discount_until': None,
  'central_billing': False,
  'city': '서울',
  'review_reply_count': 19,
  'sms_bonus': True,
  'franchise_id': None,
  'phone_downlisted': False,
  'franchise_name': None,
  'review_image_count': 204,
  'top100_restaurant': False,
  'is_available_pickup': False,
  'additional_discount_per_menu': 0,
  'delivery_fee': 1000,
  'additional_discount_pickup': 0,
  'phone': '050712913435',
  'additional_discount_only_for_user': False,
  'has_terminal': True,
  'categories': ['한식', '야식'],
  'min_order_amount': 13000,
  'distance': 1.1403644762034666,
  'logo_url': '/media/restaurant_logos/스타삼겹살-신이문역점_20170628_가맹로고이미지약정서_crop_200x200.jpg',
  'except_cash': False,
  'name': '스타삼겹살',
  'section_pos': 

In [21]:
for rest in restaurants:
    category_name = rest['categories'][0]
#     print(category_name)
    category = Category.objects.get(name=category_name) # 요기요는 M:N 관계 => 지정 name의 Category가 없다면 Category does not exist 에러 발생
    latlng = '{lat},{lng}'.format(**rest)
    logo_url = urljoin(Yogiyo.HOST, rest['logo_url'])
    shop = Shop(category=category, name=rest['name'], latlng=latlng, meta=rest)
    
    logo_name = os.path.basename(logo_url)
    logo_data = requests.get(logo_url).content
    shop.photo.save(logo_name, ContentFile(logo_data), save=False)
    print('{name}: {lat},{lng}, {categories}, {logo_url}'.format(**rest))
    
    shop.save()

한식
스타삼겹살: 37.6004127803078,127.067419293543, ['한식', '야식'], /media/restaurant_logos/스타삼겹살-신이문역점_20170628_가맹로고이미지약정서_crop_200x200.jpg
한식
김밥천국-중랑구점: 37.5992738156083,127.079625302467, ['한식', '분식', '야식'], /media/restaurant_logos/김밥세트02_20131128_FoodAD_crop_200x200_hBEzb2t.jpg
야식
왕십리할매곱창(면목동): 37.5870425,127.0784231, ['야식', '한식'], /media/restaurant_logos/곱창볶음01_20120524_FoodAD_crop_200x200_56.jpg
야식
아라곱창-본점: 37.5668464999925,127.061855142163, ['야식', '한식'], /media/restaurant_logos/곱창02_20131128_FoodAD_crop_200x200_r5SGPHa.jpg
치킨
롯데리아-외대역점: 37.5963447,127.0609306, ['치킨', '피자양식', '프랜차이즈', '야식', '1인분주문'], /media/restaurant_logos/롯데리아_20160408_Franchise이미지약정서_crop_200x200_QdMGR0r.jpg
야식
옐로우피자-안암점: 37.5867017288904,127.035797735125, ['야식', '치킨', '피자양식'], /media/restaurant_logos/페퍼로니치즈크러스트피자03_20131128_FoodAD_crop_200x200_sXen7nx.jpg
프랜차이즈
네네치킨-청량리점: 37.5848111,127.0439332, ['프랜차이즈', '야식', '치킨'], /media/restaurant_logos/네네치킨_20131201_Franchise_crop_200x200_40.jpg
중식
취홍: 37.5841135315619,127.044202

In [23]:
for shop in Shop.objects.all():
    print(shop.name, shop)

스타삼겹살 스타삼겹살
김밥천국-중랑구점 김밥천국-중랑구점
왕십리할매곱창(면목동) 왕십리할매곱창(면목동)
아라곱창-본점 아라곱창-본점
롯데리아-외대역점 롯데리아-외대역점
옐로우피자-안암점 옐로우피자-안암점
네네치킨-청량리점 네네치킨-청량리점
취홍 취홍
언제나만족 언제나만족
치킨브라더스-청량리점 치킨브라더스-청량리점
중화랑 중화랑
노랑통닭-경희대점 노랑통닭-경희대점
피자샵-장위월계점 피자샵-장위월계점
호매호 호매호
오광수산-본점 오광수산-본점
머글래싸갈래-곱창전문점 머글래싸갈래-곱창전문점
호세야오리바베큐-장안점 호세야오리바베큐-장안점
지코바치킨-청량리점 지코바치킨-청량리점
설악칡냉면 설악칡냉면
굽네치킨-이문1호점 굽네치킨-이문1호점


In [34]:
class Yogiyo:
    HOST = 'https://www.yogiyo.co.kr'
    
    def __init__(self, headers):
        self.headers = headers
        
    def get_restaurant_list_by_geo(self, zip_code):
        url = self.HOST + '/api/v1/restaurants-geo/'
        params = dict(
            items=20,
            order='rank',
            page=0,
            search='',
            zip_code=zip_code,
        )
        res = requests.get(url, params=params, headers=self.headers)
        return res.json()
    
    def get_menu_list(self, restaurant_id):
        base_url = self.HOST + '/api/v1/restaurants/{restaurant_id}/menu/?add_photo_menu=original'
        url = base_url.format(restaurant_id=restaurant_id)
        res = requests.get(url, headers=self.headers)
        return res.json()

for shop in Shop.objects.all():
    restaurant_id = shop.meta['id'] # 요기요 내에서의restaurant_id
    menu_list = yogiyo.get_menu_list(restaurant_id)
    print(shop.name)
    for sub_menu_list in menu_list:
        items = sub_menu_list['items']
        for item_meta in items:
            item = Item(shop=shop, name=item_meta['name'], amount=item_meta['price'], meta=item_meta)
            
            item_image_url = item_meta.get('image', '') # image가 없는 메뉴도 있습니다.
            if item_image_url:
                item_image_url = urljoin(Yogiyo.HOST, item_meta['image'])
                
                item_image_name = os.path.basename(item_image_url)
                item_image_data = requests.get(item_image_url).content
                item.photo.save(item_image_name, ContentFile(item_image_data), save=False)
                
            print('saving item : {}'.format(item.name))
            item.save()

NameError: name 'reuqests' is not defined